## HW7

In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima.model import ARIMAResults
from statsmodels.tsa.arima_process import ArmaProcess
from numpy.random import default_rng
import numpy as np
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings('ignore')

## Part I

a) Generate N=1000 samples of T = 100 observations from a Gaussian zero-mean AR(1) process with alpha= 0.9. Using adfuller() function with 1 lag and no constant or time trend, test each sample for unit root. Print the % of times the null (of unit root) is rejected.

In [2]:
gen = np.random.default_rng(100)

In [3]:
alpha=0.9

ar_param = np.array([alpha])
ma_param = np.array([0])
ar = np.r_[1, -alpha]
ma = np.r_[1, 0]
ar1_process = ArmaProcess(ar, ma)

n_obs = 100
n_rep=500
data = ar1_process.generate_sample(nsample = (n_obs, n_rep),  burnin=1000, distrvs=gen.standard_normal)

In [5]:
data[:3,:5]

array([[-4.89850525, -2.02602575, -1.9042407 , -1.99571436,  0.17642201],
       [-3.93123097, -3.16277086, -1.66848813, -3.11960184,  0.09681346],
       [-4.95316403, -2.67774744, -2.17262057, -2.77348177,  1.84863365]])

</div> <div class="alert alert-block alert-danger">

in python, you can replace    
    
```
number_rej_H0 = number_rej_H0 + 1
```
    
with
    
```
number_rej_H0 += 1
```
    
    
the advantage is that is is more easily readable

In [6]:
number_rej_H0=0
p_value=0.05
for c in range(0,n_rep):
    dftest = adfuller(data[:,c], maxlag=1, autolag=None, regression='n')
    if dftest[1] < p_value:
        number_rej_H0=number_rej_H0+1
percentage_rej= number_rej_H0/n_rep
print(f'The null of unit root is rejected {percentage_rej*100}% of the time.')

The null of unit root is rejected 75.6% of the time.


b) Using the code you developed for (a) write a function which accepts N, T, and alpha as parameters and prints those values (T=, N=, alpha=) and the percent of rejections of the null hypothesis

In [12]:
def test(n, t, alpha, p_value=0.05):
    gen = np.random.default_rng(100)
    ar_param = np.array([alpha])
    ma_param = np.array([0])
    ar = np.r_[1, -alpha]
    ma = np.r_[1, 0]
    ar1_process = ArmaProcess(ar, ma)

    data = ar1_process.generate_sample(nsample = (t, n),  burnin=1000, distrvs=gen.normal)
    
    number_rej_H0=0
    for c in range(0,n):
        dftest = adfuller(data[:,c], maxlag=1, autolag=None, regression='n')
        if dftest[1] < p_value:
            number_rej_H0=number_rej_H0+1
    percentage_rej= number_rej_H0/n
 
    return t, n, alpha, percentage_rej

</div> <div class="alert alert-block alert-danger">

Here you run `test(1000,100,0.9)` 3 times unnecessarily. You could have the function print that information (as it says in the task description.) by adding
```
print(f'For t={t}, n={n}, alpha={alpha}, ADF rejects the null {100*percentage_rej}% of the times')
```
 
or run the function once, assign the last output to `perc_rej` and then print.

In [8]:
print(f'Given t={test(1000,100,0.9)[0]}, N={test(1000,100,0.9)[1]} and alpha0={test(1000,100,0.9)[2]}, the null of unit root is rejected {test(1000,100,0.9)[3]*100}% of the time.')

Given t=100, N=1000 and alpha0=0.9, the null of unit root is rejected 75.9% of the time.


c) Run the function for N=1000 and all combinations of T=[100, 500, 1000], and alpha = [.9, 0.95, 0.99] and print the results as a table.

In [8]:
table=pd.DataFrame(columns=['T=100', 'T=500', 'T=1000'], index=['alpha=0.9','alpha=0.95','alpha=0.99'])

for c in {100, 500, 1000}:
    for x in {.9, 0.95, 0.99}:
        T,N,alpha,rej=test(1000,c,x);
        table.loc[f'alpha={x}',f'T={c}']=rej

print(table)

            T=100  T=500 T=1000
alpha=0.9   0.759    1.0    1.0
alpha=0.95  0.348    1.0    1.0
alpha=0.99  0.099  0.347  0.797


</div> <div class="alert alert-block alert-danger">

The last line (and the first column) in the table shows that ADF (and all other unit root) tests cannot distinguish highly persistent stationary processes from nonstationary processes very well. This is called having low power (against I(0) alternative)